In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.sparse import csr_matrix
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM
import pandas as pd
import tensorflow as tf
import torch
import pickle
from tensorflow.keras.optimizers.legacy import Adam
from tabulate import tabulate

In [8]:
df1 = pd.read_csv ('/content/drive/MyDrive/Recommendation/taskfullshare.csv')
df1 = df1.loc[:, ~df1.columns.str.contains('^Unnamed')]
df1.head()

,Init,service,user
0,1655018492726,328,1275352
1,1655018824405,92,1275354
2,1655018831538,12,1275355
3,1655018842708,149,1274893
4,1655018856558,42,8604


In [9]:
df=df1

In [10]:
mininit= df.Init.min()
df['Time']=df.Init.apply(lambda x: round((x- mininit)/2000000000))
# df.drop(columns='Init', inplace=True)
df= df[df['user'] >5 ]
df.sort_values(by='Init',ascending=False).head(2)

,Init,service,user,Time
1681001,1681064637300,395,89038,92
1681000,1681064581152,42,216925,92


In [11]:
df1['Init'] = pd.to_datetime(df1['Init'],unit='ms')
df1=df1.sort_values(by='Init',ascending=False)
df1.head(2)

,Init,service,user,Time
1681001,2023-04-09 18:23:57.300,395,89038,92
1681000,2023-04-09 18:23:01.152,42,216925,92


In [12]:
df1 = df1.sort_values(by='Init')
# Extract date and time components
df1['date'] = df1['Init'].dt.date
df1['time'] = df1['Init'].dt.time
df1.head()

,Init,service,user,Time,date,time
40557,2017-06-13 11:00:31.553,0,127,0,2017-06-13,11:00:31.553000
40555,2017-06-13 12:49:36.030,0,130,0,2017-06-13,12:49:36.030000
40554,2017-06-13 13:04:06.644,0,132,0,2017-06-13,13:04:06.644000
40597,2017-06-13 13:24:43.878,0,130,0,2017-06-13,13:24:43.878000
40629,2017-06-14 07:35:55.062,0,134,0,2017-06-14,07:35:55.062000


In [13]:
df1= df1.drop_duplicates(subset=['date', 'service', 'user'])


In [14]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1482958 entries, 40557 to 1681000
Data columns (total 6 columns):
 #   Column   Non-Null Count    Dtype         
---  ------   --------------    -----         
 0   Init     1482958 non-null  datetime64[ns]
 1   service  1482958 non-null  int64         
 2   user     1482958 non-null  int64         
 3   Time     1482958 non-null  int64         
 4   date     1482958 non-null  object        
 5   time     1482958 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(2)
memory usage: 79.2+ MB


In [15]:
df1=df1.drop(['date','time'],axis=1)
df1.head(1)

,Init,service,user,Time
40557,2017-06-13 11:00:31.553,0,127,0


In [16]:
df1.to_csv('/content/drive/MyDrive/Recommendation/final_data.csv')

In [ ]:
max_len= 10 # len(tf.keras.utils.pad_sequences(seqservice)[0])
item_vocab_size= 588

In [ ]:
# seqservice , seqtime ,seqtarget  =[] ,[] ,[] 

# for user in df1.user.unique():
#   user_data = df1[df1['user'] == user].sort_values('Time')
#   seqservice.append(user_data.service.values.tolist()[0:-1])
#   seqtime.append(user_data.Time.values.tolist()[0:-1])
#   seqtarget.append(user_data.Time.values.tolist()[1:])

#   # print(user_data)
#   # break
# train_dict = dict(zip(['item_id', 'nb_days', 'target'],[tf.keras.utils.pad_sequences(seqservice, maxlen=max_len), tf.keras.utils.pad_sequences(seqtime, maxlen=max_len), tf.keras.utils.pad_sequences(seqtarget, maxlen=max_len)]))
# # save dictionary to person_data.pkl file
# with open('/content/drive/MyDrive/Recommendation/train.pkl', 'wb') as fp:
#     pickle.dump(train_dict, fp)
#     print('dictionary saved successfully to file')
     

dictionary saved successfully to file


In [ ]:
with open('/content/drive/MyDrive/Recommendation/train_dict10.pkl', 'rb') as fp:
    train_dict = pickle.load(fp)
    print(' dictionary loaded')

 dictionary loaded


In [ ]:
#@title Create_train
def create_train_tfdata(train_feat_dict, train_target_tensor,    batch_size, buffer_size=None):
    """
    Create train tf dataset for model train input
    :param train_feat_dict: dict, containing the features tensors for train data
    :param train_target_tensor: np.array(), the training TARGET tensor
    :param batch_size: (int) size of the batch to work with
    :param buffer_size: (int) Optional. Default is None. Size of the buffer
    :return: (tuple) 1st element is the training dataset,
                     2nd is the number of steps per epoch (based on batch size)
    """
    if buffer_size is None:
        buffer_size = batch_size*50

    train_steps_per_epoch = len(train_target_tensor) // batch_size
    train_dataset = tf.data.Dataset.from_tensor_slices((train_feat_dict,train_target_tensor)).cache()
    train_dataset = train_dataset.shuffle(buffer_size).batch(batch_size)
    train_dataset = train_dataset.repeat().prefetch(tf.data.experimental.AUTOTUNE)
    
    return train_dataset, train_steps_per_epoch
  
train_feat_dict = {'item_id': train_dict['item_id'],'nb_days': train_dict['nb_days']}
train_target_tensor = train_dict['target'] 
train_dataset, train_steps_per_epoch = create_train_tfdata(train_feat_dict,  train_target_tensor, batch_size=512)

In [ ]:
#@title BuildModel
def build_model(max_len, item_vocab_size):
    """
    Build a model given the hyper-parameters with item and nb_days input features
    :param hp: (kt.HyperParameters) hyper-parameters to use when building this model
    :return: built and compiled tensorflow model 
    """
    inputs = {}
    inputs['item_id'] = tf.keras.Input(batch_input_shape=[None, max_len],   name='item_id', dtype=tf.int32)
    # create encoding padding mask
    encoding_padding_mask = tf.math.logical_not(tf.math.equal(inputs['item_id'], 0))

    # nb_days bucketized
    inputs['nb_days'] = tf.keras.Input(batch_input_shape=[None, max_len],  name='nb_days', dtype=tf.int32)

    # Pass categorical input through embedding layer
    # with size equals to tokenizer vocabulary size
    # Remember that vocab_size is len of item tokenizer + 1
    # (for the padding '0' value)
    
    embedding_item = tf.keras.layers.Embedding(input_dim=item_vocab_size,
                                               output_dim=16,
                                               name='embedding_item'
                                              )(inputs['item_id'])
    # nbins=100, +1 for zero padding
    embedding_nb_days = tf.keras.layers.Embedding(input_dim=100 + 1,
                                                  output_dim=100,
                                                  name='embedding_nb_days'
                                                 )(inputs['nb_days'])

    #  Concatenate embedding layers
    concat_embedding_input = tf.keras.layers.Concatenate(
     name='concat_embedding_input')([embedding_item, embedding_nb_days])

    concat_embedding_input = tf.keras.layers.BatchNormalization(
     name='batchnorm_inputs')(concat_embedding_input)
    
    # LSTM layer
    rnn = tf.keras.layers.LSTM(units=100,
                                   return_sequences=True,
                                   stateful=False,
                                   recurrent_initializer='glorot_normal',
                                   name='LSTM_cat'
                                   )(concat_embedding_input)

    rnn = tf.keras.layers.BatchNormalization(name='batchnorm_lstm')(rnn)

    # Self attention so key=value in inputs
    att = tf.keras.layers.Attention(use_scale=False, causal=True,
                                    name='attention')(inputs=[rnn, rnn],
                                                      mask=[encoding_padding_mask,
                                                            encoding_padding_mask])

    # Last layer is a fully connected one
    x = tf.keras.layers.Dense(item_vocab_size*2)(att)
    # x = tf.keras.layers.Dense(32)(x)
    output = tf.keras.layers.Dense(item_vocab_size, name='output')(x) #item_vocab_size

    model = tf.keras.Model(inputs, output)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(0.005),
        loss=loss_function,
        metrics=['sparse_categorical_accuracy'])
    
    return model

In [ ]:


#@title BuildModelHP
def build_modelhp(max_len, item_vocab_size , embedding_item , embedding_nb_days , rnn_units_cat , learning_rate):
    """
    Build a model given the hyper-parameters with item and nb_days input features
    :param hp: (kt.HyperParameters) hyper-parameters to use when building this model
    :return: built and compiled tensorflow model 
    """
    inputs = {}
    inputs['item_id'] = tf.keras.Input(batch_input_shape=[None, max_len],
                                       name='item_id', dtype=tf.int32)
    # create encoding padding mask
    encoding_padding_mask = tf.math.logical_not(tf.math.equal(inputs['item_id'], 0))

    # nb_days bucketized
    inputs['nb_days'] = tf.keras.Input(batch_input_shape=[None, max_len],
                                       name='nb_days', dtype=tf.int32)

    # Pass categorical input through embedding layer
    # with size equals to tokenizer vocabulary size
    # Remember that vocab_size is len of item tokenizer + 1
    # (for the padding '0' value)
    
    embedding_item = tf.keras.layers.Embedding(input_dim=item_vocab_size,
                                               output_dim=embedding_item,
                                               name='embedding_item'
                                              )(inputs['item_id'])
    # nbins=100, +1 for zero padding
    embedding_nb_days = tf.keras.layers.Embedding(input_dim=100 + 1,
                                                  output_dim=embedding_nb_days,
                                                  name='embedding_nb_days'
                                                 )(inputs['nb_days'])

    #  Concatenate embedding layers
    concat_embedding_input = tf.keras.layers.Concatenate(
     name='concat_embedding_input')([embedding_item, embedding_nb_days])

    concat_embedding_input = tf.keras.layers.BatchNormalization( name='batchnorm_inputs')(concat_embedding_input)
    
    # LSTM layer
    rnn = tf.keras.layers.LSTM(units=rnn_units_cat, return_sequences=True,  stateful=False,recurrent_initializer='glorot_normal', name='LSTM_cat')(concat_embedding_input)

    rnn = tf.keras.layers.BatchNormalization(name='batchnorm_lstm')(rnn)

    # Self attention so key=value in inputs
    att = tf.keras.layers.Attention(use_scale=False, causal=True,
                                    name='attention')(inputs=[rnn, rnn],
                                                      mask=[encoding_padding_mask,
                                                            encoding_padding_mask])

    # Last layer is a fully connected one
    output = tf.keras.layers.Dense(item_vocab_size, name='output')(att)

    model = tf.keras.Model(inputs, output)
    model.compile( optimizer=tf.keras.optimizers.Adam(learning_rate),   loss=loss_function, metrics=['sparse_categorical_accuracy'])
    
    return model
     


     


In [ ]:
#@title LossFun
def loss_function(real, pred):
    """
    We redefine our own loss function in order to get rid of the '0' value
    which is the one used for padding. This to avoid that the model optimize itself
    by predicting this value because it is the padding one.
    
    :param real: the truth
    :param pred: predictions
    :return: a masked loss where '0' in real (due to padding)
                are not taken into account for the evaluation
    """

    # to check that pred is numric and not nan
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_object_ = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,
                                                                 reduction='none')
    loss_ = loss_object_(real, pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

In [ ]:
#@title FitModel
def fit_model(model, train_dataset, steps_per_epoch, epochs):
    """
    Fit the Keras model on the training dataset for a number of given epochs
    :param model: tf model to be trained
    :param train_dataset: (tf.data.Dataset object) the training dataset
                          used to fit the model
    :param steps_per_epoch: (int) Total number of steps (batches of samples) before 
                            declaring one epoch finished and starting the next epoch.
    :param epochs: (int) the number of epochs for the fitting phase
    :return: tuple (mirrored_model, history) with trained model and model history
    """
        # mirrored_strategy allows to use multi GPUs when available
    mirrored_strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy(
        tf.distribute.experimental.CollectiveCommunication.AUTO)
    
    with mirrored_strategy.scope():
        mirrored_model = model
    early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

    history = mirrored_model.fit(train_dataset,  steps_per_epoch=steps_per_epoch,epochs=epochs,callbacks=early_stopping, verbose=2)

    return mirrored_model, history
     

In [ ]:
# #@title TrainHP
# from kerastuner.tuners import RandomSearch

# # create a tuner object and define the hyperparameter search space
# tuner = RandomSearch(
#     build_modelhp,
#     objective='val_loss',
#     max_trials=10,
#     directory='my_dir',
#     project_name='my_project',
#     hyperparameters={
#         'max_len': [3,5,10],
#         'item_vocab_size':600,
#         'embedding_item': [16, 32, 64],
#         'embedding_nb_days': [32, 64, 128],
#         'rnn_units_cat': [32, 64, 128],       
#         'learning_rate': [0.2, 0.3, 0.4]
#     })

# # prepare the training and validation data
# ...

# # start the hyperparameter search
# tuner.search(train_dataset, epochs=10)

# # fit_model(model, train_dataset, steps_per_epoch, epochs)
# # print the best hyperparameters found
# best_hyperparams = tuner.get_best_hyperparameters()[0].values
# print("Best Hyperparameters:", best_hyperparams)


In [ ]:

from keras.callbacks import EarlyStopping
#@title Train

hyperparameters={
        'max_len': max_len,
        'item_vocab_size':item_vocab_size,
        'embedding_item': 16,
        'embedding_nb_days': 16,
        'rnn_units_cat': 128,       
        'learning_rate': 0.008    }


model=build_modelhp(**hyperparameters)

fit_model(model, train_dataset,50, 50)
     


Instructions for updating:
use distribute.MultiWorkerMirroredStrategy instead


Epoch 1/50


50/50 - 16s - loss: 0.8869 - sparse_categorical_accuracy: 0.2743 - 16s/epoch - 319ms/step
Epoch 2/50


50/50 - 3s - loss: 0.7146 - sparse_categorical_accuracy: 0.3543 - 3s/epoch - 67ms/step
Epoch 3/50


50/50 - 3s - loss: 0.6601 - sparse_categorical_accuracy: 0.3730 - 3s/epoch - 54ms/step
Epoch 4/50


...

50/50 - 1s - loss: 0.3350 - sparse_categorical_accuracy: 0.4814 - 696ms/epoch - 14ms/step
Epoch 48/50


50/50 - 1s - loss: 0.2984 - sparse_categorical_accuracy: 0.5100 - 669ms/epoch - 13ms/step
Epoch 49/50


50/50 - 1s - loss: 0.2180 - sparse_categorical_accuracy: 0.5848 - 735ms/epoch - 15ms/step
Epoch 50/50


50/50 - 1s - loss: 0.1783 - sparse_categorical_accuracy: 0.6315 - 1s/epoch - 24ms/step


(<keras.engine.functional.Functional at 0x7fbb820bd1b0>,
 <keras.callbacks.History at 0x7fbb8212c220>)

******************************************************************************
# Save and load model

In [16]:
max_len= 10 # len(tf.keras.utils.pad_sequences(seqservice)[0])
item_vocab_size= 588

In [17]:
from tensorflow.keras.models import Sequential, save_model, load_model

In [ ]:
filepath = '/content/drive/MyDrive/Recommendation/Functional2'
save_model(model, filepath)

In [18]:
#@title LossFun
def loss_function(real, pred):
    """
    We redefine our own loss function in order to get rid of the '0' value
    which is the one used for padding. This to avoid that the model optimize itself
    by predicting this value because it is the padding one.
    
    :param real: the truth
    :param pred: predictions
    :return: a masked loss where '0' in real (due to padding)
                are not taken into account for the evaluation
    """

    # to check that pred is numric and not nan
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_object_ = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,
                                                                 reduction='none')
    loss_ = loss_object_(real, pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)
# # Save the model
filepath = '/content/drive/MyDrive/Recommendation/Functional'
model_S = tf.keras.models.load_model(filepath, custom_objects={'loss_function': loss_function})

In [6]:
import pandas as pd

In [7]:
servicesdf = pd.read_csv ('/content/drive/MyDrive/Recommendation/final_data.csv', index_col='Unnamed: 0')
servicesdf.head()

,Init,service,user,Time
40557,2017-06-13 11:00:31.553,0,127,0
40555,2017-06-13 12:49:36.030,0,130,0
40554,2017-06-13 13:04:06.644,0,132,0
40629,2017-06-14 07:35:55.062,0,134,0
40553,2017-06-14 07:37:21.488,4,134,0


In [ ]:
servicesdf.user.nunique()

426026

In [8]:
import csv

In [9]:
df1=pd.read_csv('/content/drive/MyDrive/Recommendation/final_data.csv')


In [10]:

def ppredict(userno):
  dftest= df1[df1.user==userno]
  dfuserservice = pd.merge(dftest, servicesdf, on="service")
  dfuserservice.drop(columns='service', inplace= True)
  # print(tabulate(dftest, headers='keys', tablefmt='psql'))


  seqservicet , seqtimet ,seqtargett  =[] ,[] ,[] 
  for user in dftest.user.unique():
    user_data = dftest[dftest['user'] == user].sort_values('Time')
    seqservicet.append(user_data.service.values.tolist()[0:-1])
    seqtimet.append(user_data.Time.values.tolist()[0:-1])
    seqtargett.append(user_data.Time.values.tolist()[1:])

  test_dict = dict(zip(['item_id', 'nb_days', 'target'],[tf.keras.utils.pad_sequences(seqservicet , maxlen=max_len), tf.keras.utils.pad_sequences(seqtimet, maxlen=max_len), tf.keras.utils.pad_sequences(seqtargett, maxlen=max_len)]))
  test_feat_dict = {'item_id': test_dict['item_id'],'nb_days': test_dict['nb_days']}
  y=model_S.predict(test_feat_dict)
  mmax=0
  mmlist=[]
  for i in range(max_len):
    values= y[0][i].tolist()
    while True:
      if max(values) in mmlist: 
        values[values.index(max(values))]=0
      else:      
        break
    mmax=max(values)
    mmlist.append(mmax)
    # print(servicesdf[servicesdf.service==values.index(max(values))].service.values)

  finaly= np.zeros((item_vocab_size))
  for i in range(len(y[0])):
      finaly+=y[0][i]
  sortyindex=np.argsort(finaly)[-10:]
  finaly.sort()
  # print(finaly[-10:])
  print('-------------------------------------- Method 2:')
  x= servicesdf[servicesdf.service.isin(sortyindex)].service.values[-10:]
  return x


************************************************************************************************************************************************************************************************************************************************

# Matrix

In [ ]:
def Recommender_System1(userid):
    #use set to remove repeated items
    Rec_Rnn=set(ppredict(userid))
    #load the dataset without time for matrix correlations recommender system
    df3= pd.read_csv ('/content/drive/MyDrive/Recommendation/dfn3share.csv')
    df3 = df3.loc[:, ~df3.columns.str.contains('^Unnamed')]
    #Create the matrix
    user = df3.groupby('user')['service'].count()
    # can filter the number of services which is used by each user
    subcat_more20 = user[user >1].index.to_list()
    df_subcat = df3[df3['user'].isin(subcat_more20)].reset_index(drop=True)
    # create a matrix from pivot tabel
    matrix1 = df_subcat.pivot_table(index=['user'], columns='service', values='count').fillna(0)

    merged_filename = f"merged_output_M{userid}.csv"
    with open(merged_filename, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['correlations', 'service'])  # Write header row
        
        for i in Rec_Rnn:
            Subcat = matrix1[i]
            Undo_music1 = pd.DataFrame(matrix1.corrwith(Subcat), columns=['correlations']).sort_values(by=['correlations'], ascending=False).head(5)
            Undo_music1['service'] = Undo_music1.index  # Add a new column "service" with the index values
            for row in Undo_music1.iterrows():
                writer.writerow(row[1])  # Write each row directly to the CSV file

    # Load merged dataset
    merged_df = pd.read_csv(merged_filename)

    # Find duplicated rows
    duplicated_rows = merged_df[merged_df.duplicated(subset=['service'])]

    # Save the duplicated rows to CSV
    drop_filename = f"/content/drive/MyDrive/Recommendation/Recommend_MA{userid}.csv"
    duplicated_rows.to_csv(drop_filename, index=False)

    return duplicated_rows


In [ ]:
Recommender_System1(135)

1/1 [==============================] - 0s 21ms/step
-------------------------------------- Method 2:


,correlations,service
20,1.000000,63.0
21,0.109595,99.0
23,0.081614,100.0


********************************************************************************
# Apriori model

In [11]:
from mlxtend.frequent_patterns import apriori


In [12]:
def Apriori(userid):
  Rec_Rnn=set(ppredict(userid))
  df = pd.read_csv ('/content/drive/MyDrive/Recommendation/taskfullshare.csv')
  servicesdf = pd.read_csv ('/content/drive/MyDrive/Recommendation/final_data.csv', index_col='Unnamed: 0')
  df=df.iloc[:,[1,2,3]]

    #حذف یوزرهایی که کمتر از دو سرویس را استفاده کرده اند
  df33=df.groupby(by=["user"]).count()
  df3=df33[df33.service<3]
  df6=df[~df.user.isin(df3.index)]
  tab4=pd.pivot_table(df6, index=['user'],columns=['service'], aggfunc=np.sum)
  tab4.columns = tab4.columns.droplevel()
  tab=tab4.fillna(0).astype(int)
  tab=tab.applymap(lambda x: 0 if x==0 else 1)  
  support=apriori(tab, min_support=0.01, use_colnames=True)
  bestprod=support.sort_values(by='support', ascending=False).head(5)
  support['servicecount']= support.itemsets.apply(lambda x : 35 in x) 

  supp2= support[support.itemsets.str.len()>1].sort_values('itemsets' , ascending=False)
  supp3=support[support.itemsets.str.len()>2].sort_values('itemsets' , ascending=False)
  supp2['service'] = supp2.itemsets.apply(lambda x: servicesdf[servicesdf.service.isin(x)].service.values)
  supp2.sort_values(by='support',ascending=False, inplace=True)
  


  merged_filename = f"merged_output_A{userid}.csv"
  with open(merged_filename, 'w', newline='') as file:
      writer = csv.writer(file)
      writer.writerow(['correlations', 'service'])  # Write header row
      # the output of RNN gives to Apriori model
      for i in Rec_Rnn:
          # Conver Apriori to dataframe
          Subcat = pd.DataFrame(supp2[supp2['service'].apply(lambda x: i in x)]).head(5)
          
          for row in Subcat.iterrows():
              writer.writerow(row[1])  # Write each row directly to the CSV file

  # Load merged dataset
  merged_df = pd.read_csv(merged_filename)

  # Find duplicated rows
  duplicated_rows = merged_df[merged_df.duplicated(subset=['service'])]

  # Save the duplicated rows to CSV
  drop_filename = f"/content/drive/MyDrive/Recommendation/Recommend_AP{userid}.csv"
  duplicated_rows.to_csv(drop_filename, index=False)

  return duplicated_rows



In [20]:
# چون دیتاست های ورودی تکراری است 
Apriori(3646)

1/1 [==============================] - 0s 25ms/step
-------------------------------------- Method 2:


,,correlations,service
0.023233,"frozenset({96, 92})",False,[92 92 92 ... 92 92 92]
